In [5]:
using Pkg

Pkg.activate("../Project.toml")

using Random
import .Iterators: cycle, Cycle, take

using JLD2
using CUDA
using Statistics
using Random

import Base: length, size, iterate, eltype, IteratorSize, IteratorEltype, haslength, @propagate_inbounds, repeat, rand, tail
using Plots; default(fmt=:png,ls=:auto)

import Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!
using Knet

 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`


In [9]:
include("../coco2014.jl")

od = load_data("../overfit_data.jld2");


┌ Info: Loaded: ../overfit_data.jld2.
└ @ Main /home/bcs/Desktop/MSc/repos/yolo-julia/coco2014.jl:43


In [11]:
dtrn = minibatch(od.x[:, :, :, 1:90], od.y[1:90], 8; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(od.x[:, :, :, 90:end], od.y[90:end], 2; xsize = (416,416,3,:), xtype=Knet.atype());


In [94]:
include("../models.jl")

Random.seed!(1)
darknet = Darknet("../yolov3.cfg"; verbose=false);

xx, yy = first(dtrn)

# @diff a = darknet(xx, yy)

out = darknet(xx, training=false);

In [95]:
out

85×10647×8 Array{Float32,3}:
[:, :, 1] =
  47.5146     47.2927     47.3645    …  419.981     419.986     419.99
  47.6526     79.5382    111.347        404.007     412.004     420.006
 115.335     108.968     109.148         33.1255     33.1212     33.0415
  91.4349     90.7924     89.0106        23.0235     22.982      22.9371
   0.513816    0.526001    0.526607       0.499768    0.499555    0.49948
   0.503642    0.506697    0.503767  …    0.499128    0.499515    0.500062
   0.483603    0.480376    0.483224       0.501414    0.50123     0.501331
   0.500737    0.487226    0.476184       0.498629    0.498082    0.49864
   0.510857    0.502731    0.504073       0.502837    0.50281     0.501518
   0.494237    0.492069    0.498524       0.501146    0.500356    0.500282
   0.505522    0.506456    0.509186  …    0.499995    0.499869    0.499591
   0.493455    0.492436    0.490958       0.501484    0.501011    0.500843
   0.503526    0.506805    0.506139       0.500721    0.500618    0.5008

In [84]:
 io[3:4,:,:,:,:] .* a.anchor_wh

2×13×13×3×8 Array{Float32,5}:
[:, :, 1, 1, 1] =
 3.60423  3.40524  3.41088  3.35801  …  3.3771   3.37235  3.39525  3.34229
 2.85734  2.83726  2.78158  2.7656      2.91991  2.90466  2.97171  2.9583

[:, :, 2, 1, 1] =
 3.44188  3.1362   3.14773  2.99527  …  2.74712  2.8572   3.0786   3.19865
 2.84878  2.85968  2.72787  2.70724     2.99213  3.03066  3.04809  3.12339

[:, :, 3, 1, 1] =
 3.28344  2.90497  2.87327  2.78193  …  2.65801  2.71235  2.94469  3.13758
 2.88105  2.87411  2.83752  2.81447     2.96312  2.90633  2.99433  3.17045

...

[:, :, 11, 1, 1] =
 3.05521  2.5704   2.42416  2.37592  …  2.43641  2.52313  2.70101  3.10989
 2.80696  2.90459  2.98384  3.0668      3.08443  3.01309  3.02538  3.20382

[:, :, 12, 1, 1] =
 3.16335  2.7159   2.6448   2.6142   …  2.69615  2.75924  2.84884  3.32541
 2.68754  2.75225  2.90986  2.97683     3.0285   2.95743  3.05728  3.14381

[:, :, 13, 1, 1] =
 3.35775  3.19512  3.0912   3.08139  …  3.11455  3.18195  3.2035   3.42747
 2.667    2.70421  2.6969

In [71]:
io

85×13×13×3×8 Array{Float32,5}:
[:, :, 1, 1, 1] =
  1.48483      2.4779       3.48014     …  12.4843       13.493
  1.48914      1.48557      1.47958         1.4817        1.49057
  0.994269     0.939376     0.940933        0.93662       0.922011
  1.01594      1.0088       0.989007        1.05661       1.05184
  0.0552783    0.104096     0.106528        0.0546616     0.0406392
  0.0145667    0.0267888    0.0150679   …   0.0435898     0.0154051
 -0.0656113   -0.078538    -0.0671293      -0.0300054    -0.00242152
  0.00294889  -0.0511077   -0.0953346      -0.0531462    -0.0610397
  0.0434359    0.0109259    0.0162909      -0.0188504    -0.0414166
 -0.0230525   -0.0317251   -0.00590487      0.00950247    0.0177049
  0.0220898    0.0258262    0.036747    …   0.0542957     0.0419495
 -0.0261819   -0.0302575   -0.0361709       0.0321826     0.0369922
  0.0141037    0.0272204    0.0245573       0.0188398     0.00871707
  ⋮                                     ⋱                
 -0.021124    -0

In [56]:
include("../utils/utils.jl")
io = convert_yolo_to_targets(a)


85×13×13×3×8 Array{Float32,5}:
[:, :, 1, 1, 1] =
  1.48483      2.4779       3.48014     …  12.4843       13.493
  1.48914      1.48557      1.47958         1.4817        1.49057
 -0.00574701  -0.0625397   -0.0608833      -0.0654781    -0.0811983
  0.015818     0.00876576  -0.0110542       0.0550641     0.0505407
  0.0552783    0.104096     0.106528        0.0546616     0.0406392
  0.0145667    0.0267888    0.0150679   …   0.0435898     0.0154051
 -0.0656113   -0.078538    -0.0671293      -0.0300054    -0.00242152
  0.00294889  -0.0511077   -0.0953346      -0.0531462    -0.0610397
  0.0434359    0.0109259    0.0162909      -0.0188504    -0.0414166
 -0.0230525   -0.0317251   -0.00590487      0.00950247    0.0177049
  0.0220898    0.0258262    0.036747    …   0.0542957     0.0419495
 -0.0261819   -0.0302575   -0.0361709       0.0321826     0.0369922
  0.0141037    0.0272204    0.0245573       0.0188398     0.00871707
  ⋮                                     ⋱                
 -0.021124   

In [54]:
aaa = cat(Y, X, dims=3)
aaa = permutedims(aaa, (3, 1, 2))
aaa = reshape(aaa, (2, 13, 13, 1, 1))

2×13×13×1×1 Array{Int64,5}:
[:, :, 1, 1, 1] =
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  1  1  1  1  1  1  1  1   1   1   1   1

[:, :, 2, 1, 1] =
 1  2  3  4  5  6  7  8  9  10  11  12  13
 2  2  2  2  2  2  2  2  2   2   2   2   2

[:, :, 3, 1, 1] =
 1  2  3  4  5  6  7  8  9  10  11  12  13
 3  3  3  3  3  3  3  3  3   3   3   3   3

...

[:, :, 11, 1, 1] =
  1   2   3   4   5   6   7   8   9  10  11  12  13
 11  11  11  11  11  11  11  11  11  11  11  11  11

[:, :, 12, 1, 1] =
  1   2   3   4   5   6   7   8   9  10  11  12  13
 12  12  12  12  12  12  12  12  12  12  12  12  12

[:, :, 13, 1, 1] =
  1   2   3   4   5   6   7   8   9  10  11  12  13
 13  13  13  13  13  13  13  13  13  13  13  13  13

In [55]:
a[1][1:2,:,:,:,:] .+ aaa

2×13×13×3×8 Array{Float32,5}:
[:, :, 1, 1, 1] =
 0.939311  1.91153   2.92053   3.90205   …  10.9182    11.9371    12.9719
 0.956565  0.942256  0.918283  0.899813      0.940603   0.926758   0.962264

[:, :, 2, 1, 1] =
 0.914492  1.86761  2.92181  3.90407  …  10.8559   11.8987   12.9547
 1.95211   1.92735  1.91805  1.91764      1.91787   1.92175   1.97037

[:, :, 3, 1, 1] =
 0.913478  1.85647  2.90455  3.87786  …  10.8486   11.9046   12.9503
 2.91947   2.90653  2.9048   2.91925      2.90278   2.89763   2.97048

...

[:, :, 11, 1, 1] =
  0.89441   1.80369   2.83741  …   9.78195  10.8141  11.8725  12.9009
 10.7943   10.8139   10.8017      10.8579   10.8725  10.9033  11.0066

[:, :, 12, 1, 1] =
  0.908456   1.83369   2.80676  …   9.80533  10.8296  11.8572  12.9084
 11.8247    11.8685   11.861       11.8928   11.8829  11.9055  11.9886

[:, :, 13, 1, 1] =
  0.963025   1.93085   2.99518  …   9.96859  10.9712  11.9728  12.962
 12.9054    12.8713   12.9142      12.9362   12.9245  12.9655  13.003

In [42]:
aaa = cat(Y, X, dims=3)

13×13×2 Array{Int64,3}:
[:, :, 1] =
  1   1   1   1   1   1   1   1   1   1   1   1   1
  2   2   2   2   2   2   2   2   2   2   2   2   2
  3   3   3   3   3   3   3   3   3   3   3   3   3
  4   4   4   4   4   4   4   4   4   4   4   4   4
  5   5   5   5   5   5   5   5   5   5   5   5   5
  6   6   6   6   6   6   6   6   6   6   6   6   6
  7   7   7   7   7   7   7   7   7   7   7   7   7
  8   8   8   8   8   8   8   8   8   8   8   8   8
  9   9   9   9   9   9   9   9   9   9   9   9   9
 10  10  10  10  10  10  10  10  10  10  10  10  10
 11  11  11  11  11  11  11  11  11  11  11  11  11
 12  12  12  12  12  12  12  12  12  12  12  12  12
 13  13  13  13  13  13  13  13  13  13  13  13  13

[:, :, 2] =
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6

In [41]:
aaa[:, :, 1]

13×13 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13
 1  2  3  4  5  6  7  8  9  10  11  12  13

In [6]:
function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end

train! (generic function with 2 methods)

In [ ]:
iters, trnloss, trnacc, tstloss, tstacc = train!(
    darknet, dtrn, dtst; 
    period=1, iters=1000, lr=1e-3, optimizer=adam!);


Iter: 0 673.1914366115224
Iter: 1 1074.0528643629768
Iter: 2 5649.146937303976
Iter: 3 22841.72896956097
Iter: 4 37453.229944393854
Iter: 5 16170.971781398599
Iter: 6 6183.348571458078
Iter: 7 2258.7688457404483
Iter: 8 1318.6386725850537
Iter: 9 815.4784805620799
Iter: 10 634.8519304119455
Iter: 11 635.5430593911083
Iter: 12 692.8783674764632
Iter: 13 698.5651530428366
Iter: 14 666.7315221316164
Iter: 15 636.0161181607572
Iter: 16 626.4178831904043
Iter: 17 680.5378372743454
Iter: 18 711.3665289189598
Iter: 19 760.4369940945777
Iter: 20 792.5189533760331
Iter: 21 859.3620715011249
Iter: 22 908.9523521178959
Iter: 23 942.4543584319138
Iter: 24 989.4704275010932
Iter: 25 1042.8589840906316
Iter: 26 1080.9012645453756
Iter: 27 1082.4048702883179
Iter: 28 1136.3681400031123
Iter: 29 1130.7836581491342
Iter: 30 1160.6329879360578
Iter: 31 1126.5884729768743
Iter: 32 1116.7944516864961
Iter: 33 1086.9525245667317
Iter: 34 1066.753478716368
Iter: 35 1040.4395251400767
Iter: 36 1011.552044333

In [ ]:
plot(iters, log.(trnloss), label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, log.(tstloss), label="test"))